In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import xlrd

## Loading Data

#### To use this method, two files have to be saved in this direction: 

##### 1) Question Raw Data, named YYYYData.xls.  The "Report" sheet at the end MUST BE DELETED
##### 2) All Students, named YYYYStudents.xlsx.

In [3]:
def load_data(year):
    directory = './'+str(year)+'Data.xls'
    
    
    overallDF = pd.DataFrame()
    book = xlrd.open_workbook(directory) 
    num_sheets = len(book.sheets())

    sheet_names = book.sheet_names()
    sheet_names

    #GET ALL STUDENTS TO USE AS THE INDEX
    book_students = xlrd.open_workbook('./'+ str(year) + 'Students.xlsx')
    all_students = pd.read_excel('./'+ str(year) + 'Students.xlsx', 'Sheet1', header=None).iloc[:,0].tolist()
    
    for i in range(num_sheets):
        this_sheet = sheet_names[i]
        df = pd.read_excel(directory, this_sheet)
        df.set_index(df.columns[0], inplace=True)
        df = df.reindex(all_students)
        df = df.reset_index()
        overallDF = pd.concat([overallDF, df], axis=1)
    

    return overallDF

In [4]:
overalldf_2008 = load_data(2008)
overalldf_2009 = load_data(2009)
overalldf_2010 = load_data(2010)

In [5]:
overalldf_2011 = load_data(2011)

## Get Mean and Variance

#### Identify pattern of columns for each subdivision.  Then find row-wise mean and variance for those selected columns and add to end of DF

In [6]:
categories = ['ppi_initiation', 'ppi_info_gather', 'ppi_closing', 'hx_physical', 'hx_social', 'pe_handwash', 'pe_phys_check', 'pe_modesty', 'ps_personal', 'ps_rec']

def generate_cols(categories):
    col_names = []
    for cat in categories:
        col_names.append("mean_" + cat)
        col_names.append("var_" + cat)
    return col_names

column_names = generate_cols(categories)

combinedDF = pd.DataFrame(columns=column_names)

In [7]:
combinedDF

,mean_ppi_initiation,var_ppi_initiation,mean_ppi_info_gather,var_ppi_info_gather,mean_ppi_closing,var_ppi_closing,mean_hx_physical,var_hx_physical,mean_hx_social,var_hx_social,mean_pe_handwash,var_pe_handwash,mean_pe_phys_check,var_pe_phys_check,mean_pe_modesty,var_pe_modesty,mean_ps_personal,var_ps_personal,mean_ps_rec,var_ps_rec


### 2009

In [8]:
dict_2009 = {
    'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
    'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
    'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI"], 
    'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"], 
    'hx_social': ["Q9_Hx"], 
    'pe_handwash': ["Q1_PE"], 
    'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"], 
    'pe_modesty': ["Q3_PE"], 
    'ps_personal': ['Q1_PS'], 
    'ps_rec': ['Q2_PS']
}




In [9]:
def get_year_df(yearDF, year_dict):
    new_year_df = pd.DataFrame()
    new_year_df["StudentName"] = yearDF.iloc[:,0]
    for key, value in year_dict.iteritems():
        selected_cols = list(yearDF.loc[:, yearDF.columns.str.contains(('|'.join(value)))].columns)
        new_year_df["mean_" + key] = yearDF[selected_cols].mean(axis=1)
        new_year_df["var_" + key] = yearDF[selected_cols].var(axis=1)
    
    return new_year_df
    
combinedDF = pd.concat([combinedDF, get_year_df(overalldf_2009, dict_2009)])


NameError: name 'combined' is not defined

In [10]:
combinedDF.head()

,StudentName,mean_hx_physical,mean_hx_social,mean_pe_handwash,mean_pe_modesty,mean_pe_phys_check,mean_ppi_closing,mean_ppi_info_gather,mean_ppi_initiation,mean_ps_personal,...,var_hx_physical,var_hx_social,var_pe_handwash,var_pe_modesty,var_pe_phys_check,var_ppi_closing,var_ppi_info_gather,var_ppi_initiation,var_ps_personal,var_ps_rec
0,"Abbey, Ashkan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Angarita, Benjamin",2.871795,3.555556,1.333333,2.666667,1.706842,2.266667,2.807692,2.769231,1.0,...,3.282051,1.777778,4.000000,4.000000,3.901740,4.064368,3.380134,3.475113,1.111111,0.488889
2,"Aniukwu, Jideofor",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Austin, Naola",2.974359,1.333333,2.666667,3.555556,2.259649,3.333333,3.692308,3.923077,2.9,...,3.090243,4.000000,4.000000,1.777778,3.951378,2.298851,1.147125,0.307692,0.766667,0.988889
4,"Ayodele, Maranatha",2.820513,3.111111,3.555556,3.111111,2.282281,2.933333,3.692308,3.692308,2.3,...,3.369963,3.111111,1.777778,3.111111,3.856682,3.236782,1.147125,1.158371,1.344444,1.555556


## CLEAN BELOW THIS LINE

In [ ]:
def add_mean_and_var_column(question_names, new_col_name):
    selected_cols = list(overallDF.loc[:, overallDF.columns.str.contains(('|'.join(question_names)))].columns)
    overallDF["mean_" + new_col_name] = overallDF[selected_cols].mean(axis=1)
    overallDF["var_" + new_col_name] = overallDF[selected_cols].var(axis=1)

add_mean_and_var_column(ppi_initiation_qs, 'ppi_initiation')
add_mean_and_var_column(ppi_info_gather, 'ppi_info_gather')
add_mean_and_var_column(ppi_closing, 'ppi_closing')

In [ ]:
overallDF.columns.tolist()

In [ ]:
hx_physical =["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"]
hx_social = ["Q9_Hx"]

add_mean_and_var_column(hx_physical, 'hx_physical')
add_mean_and_var_column(hx_social, 'hx_social')

In [ ]:
pe_handwash = ["Q1_PE"]
pe_phys_check = ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"]
pe_modesty = ["Q3_PE"]

add_mean_and_var_column(pe_handwash, 'pe_handwash')
add_mean_and_var_column(pe_phys_check, 'pe_phys_check')
add_mean_and_var_column(pe_modesty, 'pe_modesty')